In [1]:
!pip install plotly

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import r2_score

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
# setting Jedha color palette as default
pio.templates["jedha"] = go.layout.Template(
    layout_colorway=["#4B9AC7", "#4BE8E0", "#9DD4F3", "#97FBF6", "#2A7FAF", "#23B1AB", "#0E3449", "#015955"]
)
pio.templates.default = "jedha"
pio.renderers.default = "iframe" # to be replaced by "iframe" if working on JULIE

In [3]:
# Import dataset
print("Loading dataset...")
df = pd.read_csv("Walmart_Store_sales.csv")
print("...Done.")
print()

Loading dataset...
...Done.



In [4]:
print(df.shape)
df.head()

(150, 8)


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,27-07-2012,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092


In [5]:
df = df.dropna(subset = ['Weekly_Sales', 'Date'])
df["Holiday_Flag"] = df["Holiday_Flag"].fillna(0)

In [6]:
print(df.shape)
df.head()

(118, 8)


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,0.0,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092
5,4.0,28-05-2010,1857533.70,0.0,NaN,2.756,126.160226,7.896
6,15.0,03-06-2011,695396.19,0.0,69.80,4.069,134.855161,7.658


In [7]:
df["Date"] = pd.to_datetime(df['Date'])

In [8]:
df.fillna(df.mean(), inplace = True)

<ipython-input-8-af658eae1e37>:1: FutureWarning:

DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.



In [9]:
df['year']= df['Date'].dt.year
df['month']= df['Date'].dt.month
df['day']= df['Date'].dt.day
df['day_of_month']= df['Date'].dt.weekday
df

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,year,month,day,day_of_month
0,6.0,2011-02-18,1572117.54,0.0,59.610000,3.045,214.777523,6.858000,2011,2,18,4
1,13.0,2011-03-25,1807545.43,0.0,42.380000,3.435,128.616064,7.470000,2011,3,25,4
4,6.0,2010-05-28,1644470.66,0.0,78.890000,2.759,212.412888,7.092000,2010,5,28,4
5,4.0,2010-05-28,1857533.70,0.0,60.711589,2.756,126.160226,7.896000,2010,5,28,4
6,15.0,2011-03-06,695396.19,0.0,69.800000,4.069,134.855161,7.658000,2011,3,6,6
...,...,...,...,...,...,...,...,...,...,...,...,...
144,3.0,2012-10-19,424513.08,0.0,73.440000,3.594,226.968844,6.034000,2012,10,19,4
145,14.0,2010-06-18,2248645.59,0.0,72.620000,2.780,182.442420,8.899000,2010,6,18,4
147,17.0,2010-11-06,845252.21,0.0,57.140000,2.841,126.111903,7.681299,2010,11,6,5
148,8.0,2011-12-08,856796.10,0.0,86.050000,3.638,219.007525,7.681299,2011,12,8,3


In [10]:
#Suppression de la colone Date
df = df.drop("Date", axis=1)

In [11]:
df

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,year,month,day,day_of_month
0,6.0,1572117.54,0.0,59.610000,3.045,214.777523,6.858000,2011,2,18,4
1,13.0,1807545.43,0.0,42.380000,3.435,128.616064,7.470000,2011,3,25,4
4,6.0,1644470.66,0.0,78.890000,2.759,212.412888,7.092000,2010,5,28,4
5,4.0,1857533.70,0.0,60.711589,2.756,126.160226,7.896000,2010,5,28,4
6,15.0,695396.19,0.0,69.800000,4.069,134.855161,7.658000,2011,3,6,6
...,...,...,...,...,...,...,...,...,...,...,...
144,3.0,424513.08,0.0,73.440000,3.594,226.968844,6.034000,2012,10,19,4
145,14.0,2248645.59,0.0,72.620000,2.780,182.442420,8.899000,2010,6,18,4
147,17.0,845252.21,0.0,57.140000,2.841,126.111903,7.681299,2010,11,6,5
148,8.0,856796.10,0.0,86.050000,3.638,219.007525,7.681299,2011,12,8,3


In [12]:
# deleting outliers

desc = df.describe()
desc

col_out = ["Temperature", "Fuel_Price", "CPI", "Unemployment" ]
mask = True
for col in col_out:
    q1 = desc.loc["25%", col]
    q3 = desc.loc["75%", col]
    ecart = q3 - q1
    cond1 = q1 - 3*ecart < df[col]
    cond2 = q3 + 3*ecart > df[col]
    mask = mask & cond1 & cond2


In [13]:
mask.value_counts()

True     113
False      5
dtype: int64

In [14]:
df_clean = df.loc[mask, :]
df_clean.head()

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,year,month,day,day_of_month
0,6.0,1572117.54,0.0,59.610000,3.045,214.777523,6.858,2011,2,18,4
1,13.0,1807545.43,0.0,42.380000,3.435,128.616064,7.470,2011,3,25,4
4,6.0,1644470.66,0.0,78.890000,2.759,212.412888,7.092,2010,5,28,4
5,4.0,1857533.70,0.0,60.711589,2.756,126.160226,7.896,2010,5,28,4
6,15.0,695396.19,0.0,69.800000,4.069,134.855161,7.658,2011,3,6,6


In [15]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 113 entries, 0 to 149
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         113 non-null    float64
 1   Weekly_Sales  113 non-null    float64
 2   Holiday_Flag  113 non-null    float64
 3   Temperature   113 non-null    float64
 4   Fuel_Price    113 non-null    float64
 5   CPI           113 non-null    float64
 6   Unemployment  113 non-null    float64
 7   year          113 non-null    int64  
 8   month         113 non-null    int64  
 9   day           113 non-null    int64  
 10  day_of_month  113 non-null    int64  
dtypes: float64(7), int64(4)
memory usage: 10.6 KB


In [16]:
# Separate target variable Y from features X
print("Separating labels from features...")
features_list = ["Store", "Holiday_Flag", "Temperature","Fuel_Price", "CPI",
                 "Unemployment", "month", "day_of_month"]
numeric_indices = [2,3,4,5]
categorical_indices = [0,1,6,7]
target_variable = "Weekly_Sales"

X = df_clean.loc[:,features_list]
Y = df_clean.loc[:,target_variable]

print("...Done.")
print()

print('Y : ')
print(Y.head())
print()
print('X :')
print(X.head())

Separating labels from features...
...Done.

Y : 
0    1572117.54
1    1807545.43
4    1644470.66
5    1857533.70
6     695396.19
Name: Weekly_Sales, dtype: float64

X :
   Store  Holiday_Flag  Temperature  Fuel_Price         CPI  Unemployment  \
0    6.0           0.0    59.610000       3.045  214.777523         6.858   
1   13.0           0.0    42.380000       3.435  128.616064         7.470   
4    6.0           0.0    78.890000       2.759  212.412888         7.092   
5    4.0           0.0    60.711589       2.756  126.160226         7.896   
6   15.0           0.0    69.800000       4.069  134.855161         7.658   

   month  day_of_month  
0      2             4  
1      3             4  
4      5             4  
5      5             4  
6      3             6  


In [17]:
# Divide dataset Train set & Test set 
print("Dividing into train and test sets...")
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, 
                                                    random_state=0)
print("...Done.")
print()

Dividing into train and test sets...
...Done.



In [18]:
# Convert pandas DataFrames to numpy arrays before using scikit-learn
print("Convert pandas DataFrames to numpy arrays...")
X_train = X_train.values
X_test = X_test.values
Y_train = Y_train.tolist()
Y_test = Y_test.tolist()
print("...Done")

print(X_train[0:5,:])
print(X_test[0:2,:])
print()
print(Y_train[0:5])
print(Y_test[0:2])

Convert pandas DataFrames to numpy arrays...
...Done
[[  9.           0.          85.02         2.653      214.8965756
    7.68129907   6.           4.        ]
 [ 18.           0.          52.02         2.878      132.7633548
    9.331       10.           4.        ]
 [ 19.           0.          72.83         2.932      132.5983871
    8.099        7.           4.        ]
 [ 20.           1.          28.85         3.179      204.6432267
    7.484       12.           4.        ]
 [ 20.           0.          75.17         2.808      204.5675459
    7.856        6.           4.        ]]
[[  5.           0.          62.37         3.29154206 212.5604113
    6.768       12.           5.        ]
 [  1.           0.          64.74         3.734      221.2118132
    7.348        3.           4.        ]]

[509263.28, 1029618.1, 1366395.96, 1799737.79, 1973135.87]
[301827.36, 1677472.78]


In [19]:
# Missing values
print("Imputing missing values...")
print(X_train[0:5,:])
print()
imputer = SimpleImputer(strategy="mean")
X_train[:,numeric_indices] = imputer.fit_transform(X_train[:,numeric_indices])
print("...Done!")
print(X_train[0:5,:]) 
print() 

Imputing missing values...
[[  9.           0.          85.02         2.653      214.8965756
    7.68129907   6.           4.        ]
 [ 18.           0.          52.02         2.878      132.7633548
    9.331       10.           4.        ]
 [ 19.           0.          72.83         2.932      132.5983871
    8.099        7.           4.        ]
 [ 20.           1.          28.85         3.179      204.6432267
    7.484       12.           4.        ]
 [ 20.           0.          75.17         2.808      204.5675459
    7.856        6.           4.        ]]

...Done!
[[  9.           0.          85.02         2.653      214.8965756
    7.68129907   6.           4.        ]
 [ 18.           0.          52.02         2.878      132.7633548
    9.331       10.           4.        ]
 [ 19.           0.          72.83         2.932      132.5983871
    8.099        7.           4.        ]
 [ 20.           1.          28.85         3.179      204.6432267
    7.484       12.           4.

In [20]:
# Encoding categorical features and standardizing numerical features
print("Encoding categorical features and standardizing numerical features...")
print()
print(X_train[0:5,:])

# Normalization
numeric_transformer = StandardScaler()

# OHE / dummyfication
categorical_transformer = OneHotEncoder(drop='first')

featureencoder = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_indices),    
        ('num', numeric_transformer, numeric_indices)
        ]
    )

X_train = featureencoder.fit_transform(X_train)
print("...Done")
print(X_train[0:5,:])

Encoding categorical features and standardizing numerical features...

[[  9.           0.          85.02         2.653      214.8965756
    7.68129907   6.           4.        ]
 [ 18.           0.          52.02         2.878      132.7633548
    9.331       10.           4.        ]
 [ 19.           0.          72.83         2.932      132.5983871
    8.099        7.           4.        ]
 [ 20.           1.          28.85         3.179      204.6432267
    7.484       12.           4.        ]
 [ 20.           0.          75.17         2.808      204.5675459
    7.856        6.           4.        ]]
...Done
  (0, 7)	1.0
  (0, 23)	1.0
  (0, 32)	1.0
  (0, 35)	1.5180421537696533
  (0, 36)	-1.3692391525005305
  (0, 37)	0.985420030146518
  (0, 38)	0.25945900370072345
  (1, 15)	1.0
  (1, 27)	1.0
  (1, 32)	1.0
  (1, 35)	-0.4257779558708594
  (1, 36)	-0.8792988140570402
  (1, 37)	-1.1594047439459552
  (1, 38)	2.0494444592115024
  (2, 16)	1.0
  (2, 24)	1.0
  (2, 32)	1.0
  (2, 35)	0.8000067

In [21]:
# Regression Linéaire
print("3-fold cross-validation...")
regressor = LinearRegression()
regressor.fit(X_train, Y_train)
scores = cross_val_score(regressor, X_train, Y_train, cv=3)
print('The cross-validated R2-score is : ', scores.mean())
print('The standard deviation is : ', scores.std())

3-fold cross-validation...
The cross-validated R2-score is :  0.8467698479445431
The standard deviation is :  0.0414286288035505


In [22]:
print(scores)

[0.84283453 0.89936242 0.79811259]


In [23]:
# Perform grid search
print("Grid search...")
regressor = Ridge()
# Grid of values to be tested
params = {
    'alpha': [0.0, 0.1, 0.5, 1.0] # 0 corresponds to no regularization
}
gridsearch = GridSearchCV(regressor, param_grid = params, cv = 3, verbose = 3) # cv : the number of folds to be used for CV
gridsearch.fit(X_train, Y_train)
print("...Done.")
print("Best hyperparameters : ", gridsearch.best_params_)
print("Best R2 score : ", gridsearch.best_score_)

Grid search...
Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] alpha=0.0 .......................................................
[CV] ........................... alpha=0.0, score=0.838, total=   0.0s
[CV] alpha=0.0 .......................................................
[CV] ........................... alpha=0.0, score=0.900, total=   0.0s
[CV] alpha=0.0 .......................................................
[CV] ........................... alpha=0.0, score=0.799, total=   0.0s
[CV] alpha=0.1 .......................................................
[CV] ........................... alpha=0.1, score=0.825, total=   0.0s
[CV] alpha=0.1 .......................................................
[CV] ........................... alpha=0.1, score=0.859, total=   0.0s
[CV] alpha=0.1 .......................................................
[CV] ........................... alpha=0.1, score=0.891, total=   0.0s
[CV] alpha=0.5 .......................................................
[C

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.1s finished


In [24]:
# Predictions on training set
# The model has already be re-trained on all the training set at the end of the grid search, so we can directly use it !
print("Predictions on training set...")
Y_train_pred = gridsearch.predict(X_train)
print("...Done.")
print(Y_train_pred)
print()

Predictions on training set...
...Done.
[ 439864.01740639 1133832.27251709 1350722.23888321 2016432.69606163
 1990007.70358603 2058587.95643525 2013859.96407002 1479763.5933744
  528041.56612596 2431525.10883788 1411094.32021084 2109185.03722335
 1937349.68047498 2180309.37962179 1304039.69087903 1920382.63712877
  598105.62372323 1375778.77976948 1386713.75499367  859358.33803666
 2024977.60707977  354492.33983685 1891938.41603943  423290.18220998
 1533812.22067877 1872454.59147726 1939895.53819339 1966788.10453848
 1929021.11975693  685114.05354672  548350.01364829 1147174.73026518
  434102.4790461   573321.12865628 1520858.11779237 1920693.10411832
 2392722.92312248  541739.955359   1498158.40920933 1629064.35742501
  523232.60046254  245080.16945653  480588.03979536 1480677.4366632
  787299.06820462  412536.09964115 2001275.41817108 2230807.87947752
 1721021.42910858  466162.03834194  965281.92831462 1550311.27398186
 2173902.3391097   190643.2088855  1307488.11440745  944775.23961

In [25]:
# Missing values
print("Imputing missing values...")
print(X_test[0:5,:])
print()

X_test[:,numeric_indices] = imputer.transform(X_test[:,numeric_indices])
print("...Done!")
print(X_test[0:5,:]) 
print() 

Imputing missing values...
[[  5.           0.          62.37         3.29154206 212.5604113
    6.768       12.           5.        ]
 [  1.           0.          64.74         3.734      221.2118132
    7.348        3.           4.        ]
 [  6.           0.          60.71158879   3.523      217.2706543
    6.925        8.           4.        ]
 [  8.           0.          82.92         3.554      219.0701968
    6.425        8.           4.        ]
 [  3.           0.          73.44         3.594      226.9688442
    6.034       10.           4.        ]]

...Done!
[[  5.           0.          62.37         3.29154206 212.5604113
    6.768       12.           5.        ]
 [  1.           0.          64.74         3.734      221.2118132
    7.348        3.           4.        ]
 [  6.           0.          60.71158879   3.523      217.2706543
    6.925        8.           4.        ]
 [  8.           0.          82.92         3.554      219.0701968
    6.425        8.           4.

In [26]:
# Encoding categorical features and standardizing numerical features
print("Encoding categorical features and standardizing numerical features...")
print()
print(X_test[0:5,:])

X_test = featureencoder.transform(X_test)
print("...Done")
print(X_test[0:5,:])

Encoding categorical features and standardizing numerical features...

[[  5.           0.          62.37         3.29154206 212.5604113
    6.768       12.           5.        ]
 [  1.           0.          64.74         3.734      221.2118132
    7.348        3.           4.        ]
 [  6.           0.          60.71158879   3.523      217.2706543
    6.925        8.           4.        ]
 [  8.           0.          82.92         3.554      219.0701968
    6.425        8.           4.        ]
 [  3.           0.          73.44         3.594      226.9688442
    6.034       10.           4.        ]]
...Done
  (0, 3)	1.0
  (0, 29)	1.0
  (0, 33)	1.0
  (0, 35)	0.18387471488002846
  (0, 36)	0.021194230004679016
  (0, 37)	0.9244134968803781
  (0, 38)	-0.7315036667854571
  (1, 20)	1.0
  (1, 32)	1.0
  (1, 35)	0.3234763409360288
  (1, 36)	0.984651984643526
  (1, 37)	1.1503359792208079
  (1, 38)	-0.10218259262305492
  (2, 4)	1.0
  (2, 25)	1.0
  (2, 32)	1.0
  (2, 35)	0.08618825822307333
  (

In [27]:
# Predictions on test set
print("Predictions on test set...")
Y_test_pred = gridsearch.predict(X_test)
print("...Done.")
print(Y_test_pred)
print()

Predictions on test set...
...Done.
[ 380441.84386468 1362097.17135821 1594526.54840585  781191.14035975
  505223.31867602 1182637.3336401  1908228.7406636  2374899.83148747
 2207289.39338058 1651427.89328285  734113.01092664 1943866.51709159
 1284089.79600803  152154.53549211  548990.47548623  172834.5406134
  754000.56796207  302343.41704979 1853511.8303896   628330.63937813
 1923563.5017687   514785.70685588 2129932.39602907]



In [28]:
# Print R^2 scores on train/test sets for the Ridge model with optimal value of the regularization strength
print("R2 score on training set : ", r2_score(Y_train, Y_train_pred))
print("R2 score on test set : ", r2_score(Y_test, Y_test_pred))


R2 score on training set :  0.9745268032312483
R2 score on test set :  0.9317298515003947
